In [1]:
# Imports
import pandas as pd
import numpy as np
import random
from hugchat import hugchat
from hugchat.login import Login
import re
import time


In [16]:
# Enter huggingfce credentials
login = 'Your username'
password = 'Your password'
# Sign in
sign = Login(login, password)
cookies = sign.login()

cookie_path_dir = "./cookies_snapshot"
sign.saveCookiesToDir(cookie_path_dir)

chatbot = hugchat.ChatBot(cookies=cookies.get_dict()) 

In [19]:
# Load summaries
csv_file_path = "./data/youtube_shorts_with_chatbot_summary.csv"  
shorts = pd.read_csv(csv_file_path)

In [20]:
# Agent class definition
class Agent: 
    def __init__(self, age, gender, education, self_control_capacity, automatic_approach_tendency, media_use_frequency):
        self.age = age
        self.gender = gender
        self.education = education 
        self.self_control_capacity = self_control_capacity
        self.automatic_approach_tendency = automatic_approach_tendency
        self.media_use_frequency = media_use_frequency

    def toString(self):
        return f"age: {self.age}, gender: {self.gender}, education: {self.education}, scc: {self.self_control_capacity}, aat: {self.automatic_approach_tendency}, muf: {self.media_use_frequency}"

In [21]:
# Define attribute ranges
high_to_low_range = ["high", "medium", "low"]
age_range = list(range(10, 61))
gender_range = ["male", "female"]
education_range = high_to_low_range
self_control_capacity_range = high_to_low_range
automatic_approach_tendency_range = high_to_low_range
media_use_frequency_range = high_to_low_range

# Create agent with randomized attribute values
def generate_random_agent():
    age = random.choice(age_range)
    gender = random.choice(gender_range)
    education =  random.choice(education_range)
    self_control_capacity = random.choice(self_control_capacity_range)
    automatic_approach_tendency =  random.choice(automatic_approach_tendency_range)
    media_use_frequency = random.choice(media_use_frequency_range)

    agent = Agent(age=age, gender=gender, education=education, self_control_capacity=self_control_capacity, automatic_approach_tendency=automatic_approach_tendency, media_use_frequency=media_use_frequency)
    return agent
# Generate random agent
def agents_factory(num_personas):
    agents = [generate_random_agent() for _ in range(num_personas)]
    return agents

In [22]:
# Generate agents
num_agents_to_generate = 10
agents = agents_factory(num_agents_to_generate)

# Show generated agents
for agent in agents:
    print(agent.toString())

age: 31, gender: female, education: high, scc: medium, aat: medium, muf: high
age: 40, gender: female, education: medium, scc: high, aat: low, muf: high
age: 50, gender: female, education: low, scc: medium, aat: high, muf: low
age: 54, gender: male, education: high, scc: medium, aat: medium, muf: low
age: 31, gender: male, education: medium, scc: high, aat: medium, muf: low
age: 17, gender: female, education: high, scc: low, aat: medium, muf: high
age: 49, gender: male, education: low, scc: low, aat: medium, muf: medium
age: 29, gender: female, education: low, scc: high, aat: high, muf: medium
age: 60, gender: male, education: medium, scc: high, aat: medium, muf: medium
age: 33, gender: female, education: low, scc: high, aat: low, muf: high


In [23]:
# Extracting the score as the chatbot tends to add unnecessary text
def extract_score(text):
    numbers = re.findall('[01]', text)
    return numbers

# Create prompts
def process(agent, short):

    # Agent description and additional information about terminology
    agent_prompt = f'''You are a {agent.age} year old {agent.gender} with a {agent.education} degree of education. 
    Your automatic approach tendency is {agent.automatic_approach_tendency} and your media use frequency is {agent.media_use_frequency}. 
    According to the Communication Science, automatic approach tendencies are defined as the subconscious of reflexive tendency to instinctively approaching something without deliberate thought.
    This can also be translated to the use of social media.'''

    # Short description and clarification of objective 
    short_prompt = f''' You are using Youtube and are given the option to watch the following YouTube Short: 
                Title: {short['Video Title']},
                Description: {short['LLM Summary']}
                Simultaneously, you have an important project tomorrow which needs extensive preparation.   
                Based on the information I gave you about your persona and the content of the video, you should decide whether you will watch this Short or presume working on your project. 
                Return a 1 for watching the Short and a 0 for not watching the Short. Only return this number.'''
    
    # Fuse both prompts together
    final_prompt = agent_prompt + short_prompt
    # send prompt to chatbot
    response = chatbot.query(final_prompt)
    
    # Extract relevant information from chatbot response and persona
    output = {
        "score": extract_score(str(response)),
        "age": agent.age,
        "gender": agent.gender,
        "education": agent.education,
        "scc": agent.self_control_capacity, 
        "aat": agent.automatic_approach_tendency,
        "muf": agent.media_use_frequency,
    }
    
    return output

In [24]:
# Simulate behaviour towards short from all agents 
agent_outputs = []
for agent in agents:
    agent_output = process(agent, shorts.iloc[1])
    agent_outputs.append(agent_output)
    time.sleep(10)


In [25]:
# Create dataframe for easier computation
df = pd.DataFrame(agent_outputs)
# Turn string data into numerical for calculation
map_dict = {'low': -1, 'medium': 0, 'high': 1, 'male': 0, 'female': 1}
df[['education', 'scc', 'aat', 'muf', 'gender']] = df[['education', 'scc', 'aat', 'muf', 'gender']].replace(map_dict)

# Calculate mean of attributes for agents choosing to watch short and agents choosing to not watch short
df['score'] = df['score'].apply(lambda x: int(x[0]))

# Group means together
result = df.groupby('score').mean()


In [26]:
# Resulting dataframe shows average of agents seperated on their watching behavior
result

,age,gender,education,scc,aat,muf
score,,,,,,
0,45.285714,0.428571,-0.285714,0.428571,-0.142857,-0.142857
1,25.666667,1.000000,0.333333,0.000000,0.333333,0.666667
